<a href="https://colab.research.google.com/github/ej68/561/blob/master/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors as colors
from PIL import Image
import urllib
import requests
import json
import keras


Using TensorFlow backend.


In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True

In [0]:
# Get pokemon images from pokeapi.co
# Issues with images: size not uniform, whitespace in some images is coded as 0 0 0 0 and others as 1 1 1 0 (RGBA) - either way they are transparent but the gan will get pointlessly confused? most likely idk
pokedex = {}
types = []
imagesize = 128
for i in range(1,803):
  pokemon = json.loads(requests.get("https://pokeapi.co/api/v2/pokemon/"+str(i)).text)
  # Want the following: Pokemon name, pokemon sprite ("front default"), anything we would want to condition on (e.g. type(s), generation, anything else?) (size?)
  typesJSON = pokemon.get("types")
  poketypes = []
  for j in typesJSON:
    poketypes.append(j.get("type").get("name"))

  types.append(poketypes)

  imgurl = pokemon.get("sprites").get("front_default")
  img = urllib.request.urlopen(imgurl)
  unpadded = mpimg.imread(img)
  padded = np.pad(unpadded,
      ((int((imagesize-unpadded.shape[0])/2), int((imagesize-unpadded.shape[0])/2)), 
       (int((imagesize-unpadded.shape[0])/2), int((imagesize-unpadded.shape[0])/2)), 
       (0, 0)),                      
      mode = 'constant', constant_values = 0)

if unpadded[0,0,0] == 0:
  alpha1 = 0 # Original value
  red, green, blue, alpha = padded[:,:,0], padded[:,:,1], padded[:,:,2], padded[:,:,3]
  mask = (alpha==alpha1)
  padded[:,:,0] = np.maximum(red, mask)
  padded[:,:,1] = np.maximum(green, mask)
  padded[:,:,2] = np.maximum(blue, mask)
  padded[:,:,3] = np.maximum(alpha, mask)

  np.savetxt(fname="pokemon"+str(i)+".csv", delimiter=",", X=padded.flatten())
  pokedex[i] = padded  

  if i % 100 == 0:
    plt.figure()
    plt.subplot(221)
    plt.axis('off')
    plt.imshow(pokedex.get(i))
    plt.subplot(222)
    plt.axis('off')
    plt.imshow(pokedex.get(i-25))    
    plt.subplot(223)
    plt.axis('off')
    plt.imshow(pokedex.get(i-50))    
    plt.subplot(224)
    plt.axis('off')
    plt.imshow(pokedex.get(i-75))
    plt.show()

In [0]:
def variables_from_scope(scope_name):
  return tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope_name)

# Graph Parameters
z_dimension = 64
intermediate_layer_sizes = [128*2*3, 128*32*3]
image_size = 128 * 128 * 3
batch_size = 50
iterations = 1000

def generator(z):
  hidden_layer_0 = tf.layers.dense(z, intermediate_layer_sizes[0], 
                                  name = "G0", activation = tf.nn.relu)
  hidden_layer_1 = tf.layers.dense(hidden_layer_0, intermediate_layer_sizes[1],
                                  name = "G1", activation = tf.nn.relu)
  fake_X = tf.layers.dense(hidden_layer_1, image_size, name = "G2",
                          activation = tf.sigmoid)
  return fake_X

def discriminator(image):
  hidden_layer_0 = tf.layers.dense(image, intermediate_layer_sizes[1], 
                                  name = "D0", activation = tf.nn.relu)
  hidden_layer_1 = tf.layers.dense(hidden_layer_0, intermediate_layer_sizes[0],
                                  name = "D1", activation = tf.nn.relu)
  D_output = tf.layers.dense(hidden_layer_1, 1, 
                             name = "D2", activation = tf.sigmoid)
  return D_output
 

In [0]:
tf.reset_default_graph()
# Placeholders
real_X = tf.placeholder(tf.float32, [None, image_size])
z = tf.placeholder(tf.float32, [None, z_dimension])
# Generator
with tf.variable_scope("generator") as scope:
  fake_X = generator(z)

# Discriminator
with tf.variable_scope("discriminator") as scope:
  d_on_fake = discriminator(fake_X)
  scope.reuse_variables()
  d_on_real = discriminator(real_X)
  
# Loss
g_loss = -tf.reduce_mean(tf.log(d_on_fake))
d_loss = -tf.reduce_mean(tf.log(d_on_real) + tf.log(1. - d_on_fake))

# Optimizer
optimizer = tf.train.AdamOptimizer()
g_step = optimizer.minimize(g_loss,
                           var_list = variables_from_scope("generator"))
d_step = optimizer.minimize(d_loss,
                           var_list = variables_from_scope("discriminator"))
# Initializer
initialize_all = tf.global_variables_initializer()

In [0]:
def visualize_row(images, img_dim = 128, cmap = 'RGB'):
  im = np.concatenate(np.reshape(images, [-1, img_dim, img_dim]),1)
  plt.figure()
  plt.axis("off")
  plt.imshow(im, cmap = cmap)
  plt.show()

In [0]:
def make_noise():
  return np.random.uniform(-1.0, 1.0, [batch_size, z_dimension])

def next_feed_dict():
  keys = random.sample(list(pokedex), batch_size)
  images = [pokedex.get(key) for key in keys]
  return {real_X: images,
         z: make_noise()}
          
sess = tf.Session()
sess.run(initialize_all)

for i in range(iterations):
  sess.run(g_step, feed_dict = next_feed_dict())
  sess.run(d_step, feed_dict = next_feed_dict())
  if t % 100 == 0 or t+1 == iterations:
    fake_data = sess.run(fake_X, feed_dict={z:make_noise()})
    visualize_row(fake_data[:10])